## Setup

In [20]:
from statistics import mean
import pandas as pd
import math
import numpy as np

In [2]:
df1 = pd.read_csv(r"dados\fake-news-classification\WELFake_Dataset.csv")

df1.info()
df1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72134 non-null  int64 
 1   title       71576 non-null  object
 2   text        72095 non-null  object
 3   label       72134 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB


,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [30]:
print(mean([len(i.split()) for i in df1['title'].dropna()]))
print(mean([len(i.split()) for i in df1['text'].dropna()]))

12.268218397228122
540.8433455856856


In [35]:
df1.groupby('label').size()

label
0    35028
1    37106
dtype: int64

In [38]:
df3 = pd.concat([pd.read_csv(r"dados\misinformation-fake-news-text-dataset-79k\DataSet_Misinfo_FAKE.csv"), pd.read_csv(r"dados\misinformation-fake-news-text-dataset-79k\DataSet_Misinfo_TRUE.csv")])

df3.info()
df3.head()

<class 'pandas.core.frame.DataFrame'>
Index: 78617 entries, 0 to 34974
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  78617 non-null  int64 
 1   text        78588 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.8+ MB


,Unnamed: 0,text
0,0,Donald Trump just couldn t wish all Americans ...
1,1,House Intelligence Committee Chairman Devin Nu...
2,2,"On Friday, it was revealed that former Milwauk..."
3,3,"On Christmas day, Donald Trump announced that ..."
4,4,Pope Francis used his annual Christmas Day mes...


In [40]:
print(mean([len(i.split()) for i in df3['text'].dropna()]))

479.60370540031556


In [41]:
df3 = pd.concat([pd.read_csv(r"dados\fake-news-detection-datasets\Fake.csv"), pd.read_csv(r"dados\fake-news-detection-datasets\True.csv")])

df3.info()
df3.head()

<class 'pandas.core.frame.DataFrame'>
Index: 44898 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [42]:
print(mean([len(i.split()) for i in df3['title'].dropna()]))
print(mean([len(i.split()) for i in df3['text'].dropna()]))

12.453472315025168
405.28228428883244
